In [162]:
import pandas as pd
import seaborn as sns
import numpy as np

#initial data import
df = pd.read_excel('Data.xlsx')
#want to see all columns
pd.set_option('display.max_columns', None)

### Data Exploration/Tidying

To do:

- ~~Remove individual salary columns~~
- Move total salary between term and employment
- Look into employment status columns
- ~~Split up loan purpose column into each type~~
- ~~Add multiple customers indicator column~~
- Application type and industry sector need to be encoded
- Can we use age or not?
- Can we use nationality? most are irish obviously
- Make existing mortgate to salary mandated binary
- ~~Make LT 1 year values more useable~~
- Work out what status means

In [163]:
#remove last three columns which are empty
df = df.iloc[:,:-3]

In [164]:
#combine salaries of partners
df["Total Salary"] = df.iloc[:,2]+df.iloc[:,3]

In [165]:
#count number of rejects - about half and half (should be fine for not bothering with inference)
#must address the bias issue!!!
(df["Status - G/B/R"]=="R").sum()

6344

In [166]:
#drop separated customer salary columns
df = df.drop(df.columns[2:4],axis=1)

In [167]:
#sanity check, make sure most purposes is 3 (3x3 letters + 2 slash)
max(df["Loan Purpose"].str.len())

#split up each of three purposes into their own column
new = df["Loan Purpose"].str.split("/", n = 2, expand = True)

#replace none with nan
new = new.fillna(value=np.nan)

#add new columns into our data frame corresponding to each distinct purpose
df["Purpose1"] = new[0]
df["Purpose2"] = new[1]
df["Purpose3"] = new[2]

#remove loan purpose column
df = df.drop(["Loan Purpose"],axis=1)

In [168]:
#add new variable for if there are multiple customers (i.e. married couple etc)
df['Multiple Customers'] = np.where(df['Customer2 Age (YYMM)'] == 0 , 0, 1)

In [169]:
df['Single Purpose'] = df["Purpose2"].isnull()*1

In [170]:
#check for any other non-numeric values
df["Time with Bank Years"].unique()

#also need to deal with New as well as LT 1 year

#have a look at 99 years (could be error)

array([10, 14, 6, 3, 13, 15, 11, 8, 17, 4, 9, 2, 5, 16, 20, 'New', 18, 12,
       1, 7, 19, 'LT 1 year', 22, 21, 32, 24, 99, 25, 23, 33, 26, 31, 29,
       27, 28, 30], dtype=object)

In [171]:
#indices of customers with 99 years at bank
error_idx = np.where(df["Time with Bank Years"]==99)

In [172]:
#example of column w 99 years at bank - clearly error so just remove these entries
df.iloc[1097,:]

Application  Date (YYYYMM)                        201002
Term                                                  60
Customer1 Employment Status                            F
Customer2 Emp Status                                 NaN
Application Type                                       S
Industry Sector                                       RS
Customer1 Age (YYMM)                                4405
Customer2 Age (YYMM)                                   0
National of …                                        IRE
CA Months of Unauthorised Debit Interest L6M           1
CA Average Number of Transactions L6M                 10
CA Average Balance L6M                               757
CA Average Credit Turnover L6M                      2510
Existing Mortgage                                      Y
Existing Current Account                               Y
Existing Loan                                        NaN
Existing Savings Acc                                   Y
Salary Mandated to Same Bank Cu

In [173]:
#remove 99 year rows
df = df[df["Time with Bank Years"] != 99]

In [174]:
#replace LT year with 0 years
df.loc[df["Time with Bank Years"]=="LT 1 year","Time with Bank Years"]=0
#replace New with 0 years
df.loc[df["Time with Bank Years"]=="New","Time with Bank Years"]=0
#check all value now numeric
df["Time with Bank Years"].unique()

array([10, 14, 6, 3, 13, 15, 11, 8, 17, 4, 9, 2, 5, 16, 20, 0, 18, 12, 1,
       7, 19, 22, 21, 32, 24, 25, 23, 33, 26, 31, 29, 27, 28, 30],
      dtype=object)

In [179]:
df.drop(df["Status"])

KeyError: '[nan nan nan ... nan nan nan] not found in axis'